# SQLServerの（売上、マスター情報）をBigQuery用のDBに登録する。

In [22]:
!pip install pyodbc
!pip install sqlalchemy==1.4.46
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text

# SQL Serverへの接続設定
server = 'SO-PC'
database = 'datamart'
username = 'gpkadmin'
password = '19vK8xEQ'
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=SQL+Server'

# SQLAlchemyエンジンを作成
engine = create_engine(connection_string)

# 売上

In [3]:
from sqlalchemy import Table, MetaData, Column, Integer, String, BigInteger, Float, Date, select

# メタデータオブジェクトを作成
metadata = MetaData()

table1 = Table(
    'sales', metadata,
    Column('sales_date', Date, primary_key=True),
    Column('receipt_number', BigInteger, primary_key=True),
    Column('receipt_suffix', BigInteger, primary_key=True),
    Column('department', String(2)),
    Column('branch', String(50)),
    Column('order_number', String(8)),
    Column('detail_number', BigInteger),
    Column('order_type', BigInteger),
    Column('agency_code', String(7)),
    Column('customer_code', String(10)),
    Column('product_code', String(10)),
    Column('quantity', BigInteger),
    Column('amount', BigInteger),
    Column('lbc', BigInteger),
    Column('grade', String(2)),
    Column('company', String(255)),
    Column('company_kana', String(255)),
    Column('office', String(255)),
    Column('postal_code', BigInteger),
    Column('prefecture_code', BigInteger),
    Column('city_code', BigInteger),
    Column('prefecture_name', String(255)),
    Column('city_name', String(255)),
    Column('town_name', String(255)),
    Column('block', String(255)),
    Column('address_number', String(255)),
    Column('building_room_number', String(255)),
    Column('number_of_offices', BigInteger),
    Column('capital', BigInteger),
    Column('current_fiscal_year_month', BigInteger),
    Column('current_revenue', BigInteger),
    Column('previous_revenue', BigInteger),
    Column('preprevious_revenue', BigInteger),
    Column('corporate_number', BigInteger),
    Column('industry', String(4000))
)


# テーブルを作成
metadata.create_all(engine)

In [5]:
# 既存のテーブルからデータをSELECTしてINSERT
with engine.begin() as connection:
    # INSERT INTO SELECTステートメントを実行
    connection.execute(
        """INSERT INTO sales 
            SELECT 
                CONVERT(DATE, CONVERT(VARCHAR(10), 売上年月日), 112) AS 売上年月日, 
                納品書Ｎｏ, 
                枝番2, 
                担当課,
                CASE
                    WHEN 担当課 = '3A' THEN '松本営業所' 
                    WHEN 担当課 = 'AA' THEN '新潟営業所' 
                    WHEN 担当課 = 'BA' THEN '大宮営業所' 
                    WHEN 担当課 = 'DA' THEN '仙台支店' 
                    WHEN 担当課 = 'FA' THEN '福岡支店' 
                    WHEN 担当課 = 'GA' THEN '本社' 
                    WHEN 担当課 = 'JA' THEN '鹿児島営業所' 
                    WHEN 担当課 = 'KA' THEN '大阪支店' 
                    WHEN 担当課 = 'PA' THEN '札幌営業所' 
                    WHEN 担当課 = 'RA' THEN '広島営業所' 
                    WHEN 担当課 = 'TA' THEN '東京支店' 
                    WHEN 担当課 = 'TB' THEN '東京支店' 
                    WHEN 担当課 = 'UA' THEN '岡山営業所' 
                    WHEN 担当課 = 'VA' THEN '松山営業所' 
                    WHEN 担当課 = 'WA' THEN '静岡営業所' 
                    WHEN 担当課 = 'YA' THEN '西東京営業所' 
                    WHEN 担当課 = 'ZA' THEN '金沢営業所' 
                    WHEN 担当課 = 'NA' THEN '名古屋支店' 
                    ELSE ''
                END AS 支店営業所名,
                受注Ｎｏ, 
                明細Ｎｏ, 
                受注形態, 
                代理店コード,
                得意先コード, 
                商品コード, 
                個数,
                金額, 
                LBC, 
                グレード, 
                CASE
                    WHEN 社名 = '' THEN TRIM(会社名)
                    ELSE 社名
                END AS 社名結合,
                社名カナ, 
                事業所, 
                郵便番号, 
                県コード,
                市区町村コード, 
                都道府県名, 
                市区郡町村名,
                町大字通称名, 
                字丁目, 
                番地番号,
                建物名部屋番号, 
                事業所数, 
                資本金,
                当期決算年月, 
                当期売上高, 
                前期売上高,
                前々期売上高, 
                法人マイナンバー, 
                良業種
            FROM 名寄せ_売上データ_ALL"""
    )
    

# 品名マスター

In [33]:
# 削除

# 削除対象テーブルを定義
metadata = MetaData()
table_to_drop = Table('product_master', metadata, autoload_with=engine)

# drop
table_to_drop.drop(engine)

In [34]:
# 作成

# メタデータオブジェクトを作成
metadata = MetaData()
table1 = Table(
    'product_master',  # テーブル名
    metadata,
    Column('product_code', String(10), primary_key=True),
    Column('department', String(2)),
    Column('product_name', String(40)),
    Column('color_code', String(4)),
    Column('quantity', Float),
    Column('minimum_unit', String(2)),
    Column('stock_unit', String(2)),
    Column('product_rank', String(2)),
    Column('pos_code', String),
    Column('production_rank', String(2)),
    Column('midterm_standard', Float),
    Column('standard_cost', Float),
)

# テーブルを作成
metadata.create_all(engine)

In [35]:
# 既存のテーブルからデータをSELECTしてINSERT
with engine.begin() as connection:
    # INSERT INTO SELECTステートメントを実行
    connection.execute(
        """INSERT INTO product_master
            SELECT 
                商品コード
                ,営業部課コード
                ,品名
                ,色コード
                ,入数
                ,最小単位
                ,在庫単位
                ,製品ランク
                ,ＰＯＳコード
                ,生産ランク
                ,期中基準
                ,標準原価
        FROM 商品マスター"""
    )

# 得意先マスター

In [43]:
# 削除

# 削除対象テーブルを定義
metadata = MetaData()
table_to_drop = Table('agency_master', metadata, autoload_with=engine)

# drop
table_to_drop.drop(engine)

In [44]:
# 作成

# メタデータオブジェクトを作成
metadata = MetaData()
agency_master_table = Table(
    'agency_master',  # テーブル名
    metadata,
    Column('agency_code', String(7), primary_key=True),
    Column('company_name', String(40)),
    Column('branch_office_name', String(20)),
    Column('contact_name', String(20)),
    Column('abbreviation', String(30)),
    Column('department', String(2)),
    Column('organization_code', BigInteger),
    Column('employee_code', BigInteger),
)

# テーブルを作成
metadata.create_all(engine)

In [46]:
# 既存のテーブルからデータをSELECTしてINSERT
with engine.begin() as connection:
    # INSERT INTO SELECTステートメントを実行
    connection.execute(
        """INSERT INTO agency_master
            SELECT 
                得意先コード
                ,社名
                ,支社支店名
                ,窓口名
                ,略称
                ,担当課コード
                ,組織コード
                ,社員コード
        FROM 得意先マスター"""
    )

# パラメーター(社員マスター)

In [65]:
# 削除

# 削除対象テーブルを定義
metadata = MetaData()
table_to_drop = Table('employee_master', metadata, autoload_with=engine)

# drop
table_to_drop.drop(engine)

In [67]:
# 作成

# メタデータオブジェクトを作成
metadata = MetaData()
agency_master_table = Table(
    'employee_master',  # テーブル名
    metadata,
    Column('employee_code', BigInteger), #primary_key=True),
    Column('employee_name', String(40)),
    Column('employee_department', String(40)),
)

# テーブルを作成
metadata.create_all(engine)

In [68]:
# 既存のテーブルからデータをSELECTしてINSERT
with engine.begin() as connection:
    # INSERT INTO SELECTステートメントを実行
    connection.execute(
        """INSERT INTO employee_master
            (employee_code
            ,employee_name
            ,employee_department)
            SELECT 
                [ＫＥＹ] AS employee_code
                ,[項目３] AS employee_name
                ,[項目４] AS employee_department
        FROM [パラメーター]"""
    )